In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression


df= pd.read_csv('/Users/alexandermaat/Downloads/nba_salaries.csv')
df.head()

def basic_wrangling(df, unique_threshold=.95, missing_threshold=.5, messages=True):
  import pandas as pd

  # primary keys or too many unique values
  # too much missing data
  # single value columns

  for col in df:
    missing = df[col].isna().sum()
    unique = df[col].nunique()
    rows = df.shape[0]

    if missing / rows >= missing_threshold:
      df.drop(columns=[col], inplace=True)
      if messages: print(f'Column "{col}" dropped because of too much missing data ({round(missing/rows, 2)*100}%)')
    elif unique / rows >= unique_threshold:
      if df[col].dtype in ['object', 'int64']:
        df.drop(columns=[col], inplace=True)
        if messages: print(f'Column "{col}" dropped because of too many unique values ({round(unique/rows, 2)*100}%)')
    elif unique == 1:
      df.drop(columns=[col], inplace=True)
      if messages: print(f'Column "{col}" dropped because it has only one value ({df[col].unique()[0]})')
  return df



In [5]:
basic_wrangling(df)

Column "Unnamed: 0" dropped because of too many unique values (100.0%)
Column "Player Name" dropped because of too many unique values (100.0%)
Column "Player-additional" dropped because of too many unique values (100.0%)


,Salary,Position,Age,Team,GP,GS,MP,FG,FGA,FG%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,48070014,PG,34,GSW,56,56,34.7,10.0,20.2,0.493,...,0.915,0.7,5.4,6.1,6.3,0.9,0.4,3.2,2.1,29.4
1,47345760,PG,32,LAC,34,3,22.2,4.1,9.9,0.408,...,0.681,0.4,2.3,2.7,5.2,0.8,0.4,2.4,1.7,11.4
2,47080179,PG,34,LAL/LAC,73,24,29.1,5.9,13.6,0.436,...,0.656,1.2,4.6,5.8,7.5,1.0,0.5,3.5,2.2,15.9
3,44474988,PF,38,LAL,55,54,35.5,11.1,22.2,0.500,...,0.768,1.2,7.1,8.3,6.8,0.9,0.6,3.2,1.6,28.9
4,44119845,PF,34,BRK/PHO,47,47,35.6,10.3,18.3,0.560,...,0.919,0.4,6.3,6.7,5.0,0.7,1.4,3.3,2.1,29.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,35096,SF,23,POR,4,0,22.3,1.8,5.8,0.304,...,0.000,0.8,3.0,3.8,1.0,0.5,1.3,1.0,2.3,4.3
463,32795,SG,25,CHO,5,0,5.6,0.2,1.2,0.167,...,1.000,0.2,0.6,0.8,1.0,0.0,0.4,0.2,0.0,1.0
464,32171,SG,29,IND,3,0,18.7,2.7,7.0,0.381,...,1.000,0.0,2.0,2.0,1.7,0.7,0.0,0.0,1.7,8.0
465,5849,PF,23,BRK,1,0,35.0,6.0,12.0,0.500,...,1.000,3.0,6.0,9.0,7.0,0.0,1.0,4.0,5.0,16.0


In [6]:
df.Position.value_counts()

Position
SG       115
SF        91
C         91
PF        86
PG        77
PG-SG      2
SF-SG      2
SG-PG      2
SF-PF      1
Name: count, dtype: int64

In [7]:
df.loc[df['Position'] == 'PG-SG', 'Position'] = 'PG'
df.loc[df['Position'] == 'SF-SG', 'Position'] = 'PG'
df.loc[df['Position'] == 'SG-PG', 'Position'] = 'PG'
df.loc[df['Position'] == 'SF-PF', 'Position'] = 'PG'
df.Position.value_counts() / df.shape[0]

# renaming bc too many unique values

Position
SG    0.246253
SF    0.194861
C     0.194861
PF    0.184154
PG    0.179872
Name: count, dtype: float64

In [8]:
# is there other data in the dataset where the variance is explaines (for the missing columns, ie 3P and 3PA for 3P%) so we will drop percentage data
df.drop(columns=['3P%', '2P%', 'FG%', 'FT%'], inplace=True)

# cant drop eFG%, so we will drop the row
df.dropna(inplace=True)
df.isna().sum()

Salary      0
Position    0
Age         0
Team        0
GP          0
GS          0
MP          0
FG          0
FGA         0
3P          0
3PA         0
2P          0
2PA         0
eFG%        0
FT          0
FTA         0
ORB         0
DRB         0
TRB         0
AST         0
STL         0
BLK         0
TOV         0
PF          0
PTS         0
dtype: int64

In [35]:
df.head()
# need to add dummy and specify x and y, do x and y first 
y = df.Salary
X = df.drop(columns=['Salary', 'Team', 'FGA', 'PTS', 'FG' , 'TRB', '2P', 'FTA', '3PA', 'MP', 'TOV', '2PA'])
X = pd.get_dummies(X, drop_first=True)
X.head()

,Age,GP,GS,3P,2PA,eFG%,FT,ORB,DRB,AST,STL,BLK,PF,Position_PF,Position_PG,Position_SF,Position_SG
0,34,56,56,4.9,8.8,0.614,4.6,0.7,5.4,6.3,0.9,0.4,2.1,False,True,False,False
1,32,34,3,1.0,6.7,0.457,2.3,0.4,2.3,5.2,0.8,0.4,1.7,False,True,False,False
2,34,73,24,1.2,9.7,0.481,2.8,1.2,4.6,7.5,1.0,0.5,2.2,False,True,False,False
3,38,55,54,2.2,15.3,0.549,4.6,1.2,7.1,6.8,0.9,0.6,1.6,True,False,False,False
4,34,47,47,2.0,13.4,0.614,6.5,0.4,6.3,5.0,0.7,1.4,2.1,True,False,False,False


In [36]:
model = LinearRegression().fit(X, y)
df_results = pd.DataFrame({'Coefficients':model.coef_}, index=model.feature_names_in_)
df_results.sort_values(by=['Coefficients'], ascending=False)

# multicolinarity problem, need to make more field goalas, but can't do that while also making less 2 and 3 pointers
# fix with VIF

# loop throguh every feature, calc VIF score based on all other features, drop VIF bigger than 3 

,Coefficients
BLK,3.087669e+06
Position_PG,2.704846e+06
Position_SF,1.982304e+06
3P,1.707373e+06
Position_SG,1.180890e+06
FT,1.166909e+06
2PA,9.375273e+05
DRB,8.957768e+05
Age,8.741645e+05
Position_PF,1.959329e+05


In [37]:
def vif(df):
  import pandas as pd
  from sklearn.linear_model import LinearRegression

  df_output = pd.DataFrame(columns=['VIF'])

  for col in df:
    y = df[col]
    X = df.drop(columns=[col])
    r_squared = LinearRegression().fit(X, y).score(X, y)
    vif = 1 / (1 - r_squared) # VIF = 3 is the cutoff
    df_output.loc[col] = vif

  return df_output.sort_values(by=['VIF'], ascending=False)

In [38]:
vif(X)

,VIF
2PA,6.955376
DRB,4.856007
FT,4.539671
AST,4.295256
Position_PG,3.888418
Position_SG,3.700314
ORB,3.433552
GS,3.244539
Position_SF,2.768424
PF,2.601519
